# Notebook App UI Example

This notebook demonstrates the in-notebook app interface letting you interact with a langchain app inside this notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

from pprint import PrettyPrinter
pp = PrettyPrinter()

from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY"
)

from trulens_eval import Tru

tru = Tru()
# tru.reset_database() # if needed
tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

## langchain example

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

llm = OpenAI(temperature=0.9, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=llm,
)

# Conversational app puts it all together.
app = ConversationChain(
    llm=llm,
    memory=memory
)

from langchain.prompts import PromptTemplate
from trulens_eval.instruments import instrument
instrument.method(PromptTemplate, "format")

truchain = tru.Chain(app)

In [ ]:
# Manual usage without instrumentation:
pp.pprint(app("Hello there"))
pp.pprint(app("What is your name?"))

In [ ]:
with truchain:
    # Manual usage with instrumentation:
    pp.pprint(app("Hello there"))
    pp.pprint(app("What is your name?"))

In [ ]:
from trulens_eval.appui import AppUI

# Interesting Select.App queries to look at:
# - app.memory.chat_memory.messages[:].content
# - app.memory.moving_summary_buffer
# - app.prompt.template

# Interesting Select.Record queries to look at:
# - app.memory.save_context[0].args
# - app.prompt.format.args.kwargs
# - app.prompt.format.rets
# The last two need to instrument PromptTemplate as above.

aui = AppUI(
    app=truchain,
    
    app_selectors=[
        "app.memory.chat_memory.messages[:].content",
        "app.memory.moving_summary_buffer",

        "app.prompt.template"
    ],
    record_selectors=[
        "app.memory.save_context[0].args",

        "app.prompt.format.args.kwargs",
        "app.prompt.format.rets"
    ]
)
aui.widget

## llama_index example

In [ ]:
from llama_index import VectorStoreIndex, SimpleWebPageReader

documents = SimpleWebPageReader(
    html_to_text=True
).load_data(["http://paulgraham.com/worked.html"])
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

trullama = tru.Llama(query_engine)

In [ ]:
from trulens_eval.appui import AppUI

aui = AppUI(
    app=trullama,
    
    app_selectors=[
    ],
    record_selectors=[
        "app.retriever.retrieve[0].rets[:].score",
        "app.retriever.retrieve[0].rets[:].node.text",
    ]
)
aui.widget

In [ ]:
from typing import Sequence
from ipywidgets import widgets

class AutoText(object):
    def __init__(self, completions: Sequence[str]):
        self.completions = completions

        self.input_widget = widgets.Text()
        self.completions_view = widgets.HTML()

        self.widget = widgets.VBox([self.input_widget, self.completions_view])

        self.input_widget.observe(self._input_updated, ["value"])

    def _input_updated(self, ev):
        new_val = ev['new']
        
        self.completions_view.value = ""

        for c in self.completions:
            if c.startswith(new_val):
                self.completions_view.value += f"{c}<br/>"

a = AutoText(completions=["piotr", "mardziel"])
a.widget

In [ ]:
from trulens_eval.utils.serial import all_queries


qs = list(all_queries(truchain))